<a href="https://colab.research.google.com/github/ArunK-ML/Learn---Deep-Learning---ANN-RNN-CNN-GAN/blob/main/Spam_mail_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%pip install streamlit

In [8]:
!pip install streamlit streamlit_option_menu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 17.8 MB/s eta 0:00:00


In [12]:
%%writefile SpamMail_streamlit_app_finall.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import torch
import torch.nn as nn
from torch.nn.functional import sigmoid
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ===========================
# Load ML Model & Vectorizer
# ===========================
try:
    with open("ML_model.pkl", "rb") as f:
        ml_model = pickle.load(f)
    with open("vectorizer.pkl", "rb") as f:
        vectorizer = pickle.load(f)

    ml_acc = 0.95  # replace with actual accuracy
except Exception as e:
    st.error(f"Error loading ML model/vectorizer: {e}")
    ml_model, vectorizer, ml_acc = None, None, 0.0

# ===========================
# Define DL Model (must match training!)
# ===========================
class SpamClassifierLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_dim=64, output_dim=1):
        super(SpamClassifierLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        out = self.fc(hidden[-1])
        return sigmoid(out)

# ===========================
# Load DL Model + Tokenizer
# ===========================
try:
    # Load tokenizer and max_len used in training
    with open("tokenizer.pkl", "rb") as f:
        tokenizer = pickle.load(f)
    with open("max_len.pkl", "rb") as f:
        max_len = pickle.load(f)

    vocab_size = len(tokenizer.word_index) + 1
    dl_model = SpamClassifierLSTM(vocab_size)
    dl_model.load_state_dict(torch.load("DL_model.pth", map_location="cpu"))
    dl_model.eval()

    dl_acc = 0.97  # replace with actual accuracy
except Exception as e:
    st.error(f"Error loading DL model/tokenizer: {e}")
    dl_model, tokenizer, max_len, dl_acc = None, None, 100, 0.0

# ===========================
# Streamlit UI
# ===========================
st.title("📧 Fraud / Spam Email Classifier")
st.write("Classify emails/messages using **ML (Logistic Regression)** and **DL (PyTorch LSTM)** models")

st.sidebar.header("Model Accuracies")
st.sidebar.write(f"**ML Accuracy:** {ml_acc:.2f}")
st.sidebar.write(f"**DL Accuracy:** {dl_acc:.2f}")

user_input = st.text_area("✍️ Enter an email/message to classify:")

if st.button("Classify"):
    if user_input.strip() == "":
        st.warning("Please enter some text.")
    else:
        # -----------------
        # ML Prediction
        # -----------------
        if ml_model and vectorizer:
            try:
                X_vec = vectorizer.transform([user_input])
                ml_pred = ml_model.predict(X_vec)[0]
                ml_label = "🚨 Spam" if ml_pred == 1 else "✅ Not Spam"
                st.subheader("ML Model Result")
                st.write(f"**Prediction:** {ml_label}")
            except Exception as e:
                st.error(f"Error during ML prediction: {e}")
        else:
            st.warning("ML model or vectorizer not loaded correctly.")

        # -----------------
        # DL Prediction (PyTorch)
        # -----------------
        if dl_model and tokenizer:
            try:
                # Tokenize text
                seq = tokenizer.texts_to_sequences([user_input])
                pad = pad_sequences(seq, maxlen=max_len, padding="post")
                pad_tensor = torch.tensor(pad, dtype=torch.long)

                with torch.no_grad():
                    output = dl_model(pad_tensor)
                    pred = (output.item() > 0.5)
                dl_label = "🚨 Spam" if pred else "✅ Not Spam"

                st.subheader("DL Model Result")
                st.write(f"**Prediction:** {dl_label}")
            except Exception as e:
                st.error(f"Error during DL prediction: {e}")
        else:
            st.warning("DL model or tokenizer not loaded correctly.")


Overwriting SpamMail_streamlit_app_finall.py


In [10]:
!wget -q -O - ipv4.icanhazip.com

34.141.168.32


In [13]:
!streamlit run SpamMail_streamlit_app_finall.py & npx localtunnel --port 8501




⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.141.168.32:8501

⠦⠧⠇⠏⠋your url is: https://odd-queens-behave.loca.lt
2025-09-04 13:49:58.759765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756993798.786800    5218 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756993798.795609    5218 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756993798.816481    5218 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756993798.816519

In [4]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.2 MB/s eta 0:00:00
